In [49]:
# TODO: use manual HTML features
import numpy as np
import pandas as pd
from utils import run_ML
from sklearn.metrics import f1_score, precision_score, recall_score
from itertools import groupby
import pickle
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
import os

In [2]:
from utils import run_LGB

In [3]:
import codecs
import gensim
import urllib
import logging
from pathlib import Path
from bs4 import BeautifulSoup
from numpy import load, save,  asarray
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import networkx as nx
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings("ignore")

In [4]:
# tmp_dir = "E:/Phising URL Detection/Baseline/DLM-main/DLM-main/tmp"
data_dir = "E:/Phising URL Detection/Methods/html_files/html_files"

In [5]:
doc2vec = gensim.models.doc2vec.Doc2Vec.load("E:/Phising URL Detection/Baseline/DLM-main/DLM-main/doc2vec.model")

In [6]:
from utils import generate_filename, convert, read_corpus

In [7]:
def callTag(tag, node):
    node_from = node
    node = node + 1

    for tag in tag.children:
        attr_node = 1
        if (tag.name is not None):
            nodes_list.append(convert([str(node_from), str(node)]))
            attr_list.append(str(node) + ",nname," + str(tag.name))
            attr_list.append(str(node) + ",value," + "")
            for attr in tag.attrs:
                nodes_list.append(convert([str(node), str(node) + "_" + str(attr_node)]))
                attr_list.append(str(node) + "_" + str(attr_node) + ",nname," + str(attr))
                attr_list.append(str(node) + "_" + str(attr_node) + ",value," + str(tag.get(attr)))
                attr_node = attr_node + 1
            callTag(tag, node)

In [8]:
def get_udst_inputs(url, rec_id, phishing_flag):
    node = 0
    f = codecs.open(Path(data_dir + '/' + generate_filename(rec_id)), 'r', encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(f)
    callTag(soup, node)

    nodes_list.pop(0)
    attr_list.pop(1)
    attr_list.insert(1, '1, value, ' + url)

    G = nx.Graph()
    G.add_edges_from(nodes_list)

    A = nx.adjacency_matrix(G) # N*N Adjacency matrix

    df_features = pd.DataFrame(0.0, index=G.nodes(), columns=['nname', 'value'])

    attr_val_list = []
    for x in attr_list:
        y = x.split(',')
        if y[1] == "value":
            attr_val_list.append(y[2])

    for x in attr_list:
        y = x.split(',')
        test_corpus = list(read_corpus([y[2]], tokens_only=True))
        vector = doc2vec.infer_vector(test_corpus[0])
        # print(test_corpus[0])

        if y[1] == "nname":
            df_features.loc[y[0]]["nname"] = vector

        else:
            df_features.loc[y[0]]["value"] = vector

    X = df_features.values # N*d Feature Matrix
    # y = to_categorical(phishing_flag, num_classes=2).tolist() # Label
    y = phishing_flag

    X_ = np.array([np.array(ai, dtype=np.float32) for ai in X.tolist()])

    # url_token = url_tokenizer([url])[0]
    url_token = None

    return X_, A, y, url_token, url

### Load data

In [9]:
%load_ext autoreload
%autoreload 2

In [10]:
df=pd.read_csv('E:/Phising URL Detection/Methods/Dataset/concat.csv')
df['result'] = df['result'].replace({'legitimate': 0, 'phishing': 1})
smalldata=df
labels=df['result']

In [11]:
smalldata.head(2)

,rec_id,url,result
0,579,https://www.pedromonjo.com/,0
1,765,http://forallstudodupdafor.co.nf/_about_.html,1


In [12]:
import collections
counter = collections.Counter(labels)
counter, len(labels)

(Counter({0: 1496, 1: 735}), 2231)

In [13]:
# Chia dữ liệu thành chỉ số train và test
train_idx = list(range(0, 1784))  # Từ 0 đến 1783
test_idx = list(range(1784, 2231))  # Từ 1784 đến 2230

train_data = smalldata.iloc[train_idx]
test_data = smalldata.iloc[test_idx]

# Tạo các biến chứa labels cho train và test
train_labels = labels.iloc[train_idx]
test_labels = labels.iloc[test_idx]

# Hiển thị kích thước của dữ liệu train và test
len(train_data), len(test_data), len(train_labels), len(test_labels)

(1784, 447, 1784, 447)

### Conventional Models

In [14]:
from utils import extract_features_url

In [15]:
extract_features_url("https://www.klikaj.hr/")

{'domain': 'www.klikaj.hr',
 'num_subdomains': 2,
 'contains_ip': 0,
 'path_length': 1,
 'num_path_segments': 1,
 'uses_https': 1,
 'file_extension': '',
 'count_special_characters': 6,
 'count_non_alphanumeric_characters': 6,
 'TLD': 'hr',
 'count_obfuscated_characters': 0,
 'letter_ratio_in_url': 0.7272727272727273,
 'digit_ratio_in_url': 0.0,
 'count_equals_in_url': 0,
 'NoOfAmpersandInURL': 0,
 'CharContinuationRate': 0.18181818181818182,
 'ratio_obfuscated_characters': 0.0,
 'NoOfQMarkInURL': 0}

In [16]:
## test on numerical URLs features
from utils import extract_numerical_features
phish_url = []
for link in list(smalldata.iloc[:,1]):
    url_features = extract_numerical_features(link)
    phish_url.append(list(url_features.values()))
# run_ML(np.array(phish_url), labels, "URLdataset13.2", "url_manual_numerical")

In [17]:
# Chia phish_url thành train và test theo train_idx và test_idx
train_phish_url = np.array(phish_url)[train_idx]
test_phish_url = np.array(phish_url)[test_idx]

# Kiểm tra kích thước của train_phish_url và test_phish_url
len(train_phish_url), len(test_phish_url)

(1784, 447)

In [18]:
run_LGB(train_phish_url, train_labels, test_phish_url, test_labels,"concat")

Running LightGBM...
F1 Score: 0.8207
Precision Score: 0.8244
Recall Score: 0.8174


In [ ]:

import lightgbm as lgb


### Extract graph features from URLs for PyG

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
# List of URLs
urls = list(smalldata['url'])
# Tokenization and N-grams Generation
# You can adjust ngram_range to extract different n-grams (e.g., (1, 1) for unigrams, (2, 2) for bigrams, etc.)
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 4)) #5
X_counts = vectorizer.fit_transform(urls)
# TF-IDF Transformation
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X_counts)
# Extracted Features
feature_names = vectorizer.get_feature_names_out()
X_counts_data = X_counts.toarray() # not necessary
# Train lgb
model_lgb = lgb.LGBMClassifier(verbose=-1)
model_lgb.fit(X_counts_data[train_idx], labels[train_idx])
y_predict=model_lgb.predict(X_counts_data[test_idx]) 
print(f1_score(y_predict, labels[test_idx], average='macro'))
feature_imp_gain = pd.DataFrame(sorted(zip(model_lgb.booster_.feature_importance(importance_type='gain'),
                                           feature_names), reverse=True), columns=['Value', 'Feature'])
feature_imp_split = pd.DataFrame(sorted(zip(model_lgb.booster_.feature_importance(importance_type='split'),
                                            feature_names), reverse=True), columns=['Value', 'Feature'])
top_ngrams_features = list(set(list(feature_imp_gain.iloc[:200,1]) + list(feature_imp_split.iloc[:200,1])))
cv = CountVectorizer(analyzer='char', ngram_range=(1, 4))
cv.fit(top_ngrams_features)

0.8853669795353132


CountVectorizer(analyzer='char', ngram_range=(1, 4))

In [21]:
def extract_feature_CountVectorizer(model, url):
    return model.transform([url]).toarray().flatten()

In [22]:
# extract countvectorizer feature of all URLs
phish_url_countvectorizer = []
for link in list(smalldata.iloc[:,1]):
    url_features = extract_feature_CountVectorizer(cv,link)
    phish_url_countvectorizer.append(url_features)
# run_ML(np.array(phish_url_countvectorizer), labels, "URLdataset13.2", "ngramURLfeatures")

In [24]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError
import traceback
import logging

# Run HTML features

In [32]:
## Idea: Combine with manual features from HTMLs

In [25]:
# HTML offline
import requests
from bs4 import BeautifulSoup
import re
# import whois
from urllib.parse import urlparse
from collections import Counter

# Function to parse HTML content from a URL
def get_html_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise error if the response is bad
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None

# Function to extract HTML structure features
def extract_structure_features(soup):
    features = {}
    
    features['num_divs'] = len(soup.find_all('div'))
    features['num_scripts'] = len(soup.find_all('script'))
    features['num_forms'] = len(soup.find_all('form'))
    features['num_links'] = len(soup.find_all('a'))
    features['num_iframes'] = len(soup.find_all('iframe'))
    
    # Text to HTML ratio
    text_length = len(soup.get_text())
    html_length = len(str(soup))
    features['text_to_html_ratio'] = text_length / html_length if html_length > 0 else 0
    
    return features

# Function to extract text-based features
def extract_text_features(soup):
    
    features = {}

    # Title tag content
    title = soup.title.string if soup.title and soup.title.string else ''
    features['title_length'] = len(title)

    # Visible text length
    visible_text = soup.get_text() if soup else ''
    features['visible_text_length'] = len(visible_text)

    return features

# Function to extract link-based features
# def extract_link_features(soup, base_url):
#     features = {}
    
#     all_links = soup.find_all('a', href=True)
#     internal_links = [link for link in all_links if urlparse(link['href']).netloc == urlparse(base_url).netloc]
#     external_links = [link for link in all_links if urlparse(link['href']).netloc != urlparse(base_url).netloc]
    
#     features['num_internal_links'] = len(internal_links)
#     features['num_external_links'] = len(external_links)
    
#     return features

from urllib.parse import urlparse

def extract_link_features(soup, base_url):
    features = {}
    
    try:
        all_links = soup.find_all('a', href=True)
        internal_links = []
        external_links = []
        
        for link in all_links:
            href = link['href']
            try:
                parsed_href = urlparse(href)
                parsed_base = urlparse(base_url)
                
                if parsed_href.netloc == parsed_base.netloc:
                    internal_links.append(href)
                else:
                    external_links.append(href)
            except ValueError:
                continue  # Skip invalid links
        
        features['num_internal_links'] = len(internal_links)
        features['num_external_links'] = len(external_links)
    except Exception as e:
        print(f"Error extracting link features: {e}")
        features['num_internal_links'] = 0
        features['num_external_links'] = 0
    
    return features


# Function to extract JavaScript-based features
def extract_js_features(soup):
    features = {}
    
    script_tags = soup.find_all('script')
    features['num_js_files'] = len(script_tags)
    
    # Suspicious JavaScript patterns
    suspicious_js_patterns = ['eval', 'document.write', 'window.location']
    features['suspicious_js_count'] = sum(
        any(p in script.get_text() for p in suspicious_js_patterns) for script in script_tags)
    
    return features

# Function to extract all features from a URL
def extract_features(url, rec_id):
#     html_content = get_html_content(url)
#     if not html_content:
#         return None
    
#     soup = BeautifulSoup(html_content, 'html.parser')
    try:
        f = codecs.open(Path(data_dir + '/' + generate_filename(rec_id)), 'r', encoding='utf-8', errors='ignore')
        soup = BeautifulSoup(f)
        
        # Extract all feature sets
        structure_features = extract_structure_features(soup)
        text_features = extract_text_features(soup)
        link_features = extract_link_features(soup, url)
        js_features = extract_js_features(soup)
        # domain_features = extract_domain_features(url)
        
        # Combine all features into a single dictionary
        features = {**structure_features, **text_features, **link_features, **js_features}
    except:
        print()
        print(f"Lỗi tại {rec_id}")
        features = [0]*12
    
    return features

In [26]:
extract_features('https://dressxsozswlzs4khuybvx6b7ur4a-gtr8-ja92z-p4te-yy23.4everland.app/cio.html', 7608)

{'num_divs': 0,
 'num_scripts': 1,
 'num_forms': 0,
 'num_links': 0,
 'num_iframes': 0,
 'text_to_html_ratio': 0.0,
 'title_length': 0,
 'visible_text_length': 0,
 'num_internal_links': 0,
 'num_external_links': 0,
 'num_js_files': 1,
 'suspicious_js_count': 0}

In [27]:
extract_features("http://vamoaestudiarmedicina.blogspot.com/", 1)

{'num_divs': 426,
 'num_scripts': 7,
 'num_forms': 0,
 'num_links': 121,
 'num_iframes': 0,
 'text_to_html_ratio': 0.23880716044000447,
 'title_length': 24,
 'visible_text_length': 44917,
 'num_internal_links': 39,
 'num_external_links': 79,
 'num_js_files': 7,
 'suspicious_js_count': 0}

In [44]:
smalldata.iloc[159,0]

2010

In [45]:
smalldata.iloc[159,1]

'https://www.kotikokki.net/'

In [ ]:
sma

In [28]:
manual_html_features = []
for idx in range(len(smalldata.index)):
    url = smalldata.iloc[idx, 1]
    print(idx,end=',')
    rec_id = smalldata.iloc[idx, 0]
    features = extract_features(url, rec_id)
    manual_html_features.append(list(features.values()))

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
Lỗi tại 1080


AttributeError: 'list' object has no attribute 'values'

In [22]:
type(manual_html_features[212])

list

In [25]:
smalldata.iloc[213,0]

7608

In [26]:
import pickle

# # Lưu vào file pickle
# with open("manual_html_features_concat.pkl", "wb") as f:
#     pickle.dump(manual_html_features, f)

# Mở file pickle và nạp dữ liệu vào biến manual_html_features
with open("manual_html_features_concat.pkl", "rb") as f:
    manual_html_features= pickle.load(f)

In [27]:
len(manual_html_features)

2231

In [28]:
run_LGB(np.array(manual_html_features)[train_idx], train_labels, np.array(manual_html_features)[test_idx], test_labels,"concat")

Running LightGBM...
F1 Score: 0.8457
Precision Score: 0.8451
Recall Score: 0.8463


In [30]:
ngramUrl_manualHtml_features = np.concatenate([phish_url_countvectorizer, manual_html_features], axis=1)

### Create dataset class for PyG

In [34]:
import torch
from torch_geometric.data import Data

In [1]:
if True: #don't delete this
    graphs = []
    labels_list = []
    for i in range(0,len(smalldata.index)):
    # for i in range(2):
        nodes_list = []
        attr_list = []
        print(i, end = ',')
        X, A, y, X_A, _ = get_udst_inputs(smalldata.iloc[i, 1], smalldata.iloc[i, 0], smalldata.iloc[i, 2])

        child_id, source_id = A.nonzero()
        edge_index = torch.tensor([source_id,
                                   child_id], dtype=torch.long)
        x = torch.tensor(X, dtype=torch.float)
        y = torch.tensor([labels[i]], dtype=torch.int64)
        data = Data(x=x, edge_index=edge_index, y = y)
        graphs.append(data)
        labels_list.append(labels[i])

NameError: name 'smalldata' is not defined

In [38]:
len(graphs)

2231

In [37]:
labels_list = labels

In [38]:
import pickle
# with open("graph_html_9.3", "wb") as fp:   #Pickling
#     pickle.dump(graphs, fp)
with open("graph_html_9.3", "rb") as fp:   # Unpickling
    graphs = pickle.load(fp)


In [39]:
print("Train freq: ", [len(list(group)) for key, group in groupby(sorted(labels_list))])

Train freq:  [1496, 735]


In [40]:
import torch
from torch_geometric.data import Data, Dataset

class GraphClassificationDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
        # self.labels = labels

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        graph = self.graphs[idx]
        # label = self.labels[idx]
        return graph
    
    def get(): pass

    def len(): pass

In [41]:
dataset = GraphClassificationDataset(graphs)

In [42]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.


Dataset: GraphClassificationDataset(2231):
Number of graphs: 2231
Number of features: 2
Number of classes: 2


In [43]:
# train_dataset = dataset[:int(0.8*n_samples)]
# test_dataset = dataset[int(0.8*n_samples):]
train_dataset = [dataset[idx] for idx in train_idx]
test_dataset = [dataset[idx] for idx in test_idx]

In [44]:
len(train_dataset), len(test_dataset)

(1784, 447)

In [45]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

### Build and train PyG

In [46]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)
        self.linconcat = Linear(2*hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

In [47]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        # print(data.x.shape)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0
    # for data in loader:  # Iterate in batches over the training/test dataset.
    #     out = model(data.x, data.edge_index, data.batch)  
    #     pred = out.argmax(dim=1)  # Use the class with highest probability.
    #     correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    # return correct / len(loader.dataset)  # Derive ratio of correct predictions.
    true_labels = []
    pred_labels = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        true_labels += data.y.tolist()
        pred_labels += pred.tolist()
        # print(pred_labels)
    return f1_score(true_labels, pred_labels, average='macro')

for epoch in range(0, 20):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Train F1: {train_acc:.4f}, Test F1: {test_acc:.4f}')

Epoch: 000, Train F1: 0.5733, Test F1: 0.5870
Epoch: 010, Train F1: 0.6419, Test F1: 0.6173


### GNN for dim. reduction

#### Net 1: 0.87, 0.88

In [48]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool

class GCNdimReduce(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCNdimReduce, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = self.lin1(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.lin2(x)
        
        return x
    
    def dimReduce(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        # x = x.relu()
        # x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # # 3. Apply a final classifier
        x = self.lin1(x)
        return x

#### Net 2

In [41]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool

class GCNdimReduceV2(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCNdimReduceV2, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
       
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        x = F.dropout(x, p=0.5, training=self.training)
        # 3. Apply a final classifier
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        
        return x
    
    def dimReduce(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin1(x)
        return x

#### Test model

In [50]:
n_hidden_channels = 16
model = GCNdimReduce(hidden_channels=n_hidden_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        # print(data.x.shape)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0
    true_labels = []
    pred_labels = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        true_labels += data.y.tolist()
        pred_labels += pred.tolist()
    return f1_score(true_labels, pred_labels, average='macro'), precision_score(true_labels, pred_labels, average='macro'), recall_score(true_labels, pred_labels, average='macro')


for epoch in range(0, 10):
    train()
    train_f1, train_pre, train_rec = test(train_loader)
    test_f1, test_pre, test_rec = test(test_loader)
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Train F1: {train_f1:.4f}, Test F1: {test_f1:.4f}')

Epoch: 000, Train F1: 0.4013, Test F1: 0.4016
Epoch: 001, Train F1: 0.4719, Test F1: 0.4630
Epoch: 002, Train F1: 0.5325, Test F1: 0.5409
Epoch: 003, Train F1: 0.5827, Test F1: 0.5800
Epoch: 004, Train F1: 0.6470, Test F1: 0.6260
Epoch: 005, Train F1: 0.6255, Test F1: 0.6008
Epoch: 006, Train F1: 0.6303, Test F1: 0.5990
Epoch: 007, Train F1: 0.6408, Test F1: 0.6123
Epoch: 008, Train F1: 0.6374, Test F1: 0.6086
Epoch: 009, Train F1: 0.6423, Test F1: 0.6183


#### Def

In [51]:
def train_model(all_data_loader, train_loader, test_loader, n_hidden_channels = 16, n_epoch=1, lr=0.001):
    # n_hidden_channels = 16
    model = GCNdimReduce(hidden_channels=n_hidden_channels)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    def train():
        model.train()

        for data in train_loader:  # Iterate in batches over the training dataset.
            # print(data.x.shape)
            out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
            loss = criterion(out, data.y)  # Compute the loss.
            loss.backward()  # Derive gradients.
            optimizer.step()  # Update parameters based on gradients.
            optimizer.zero_grad()  # Clear gradients.

    def test(loader):
        model.eval()
        correct = 0
        true_labels = []
        pred_labels = []
        for data in loader:  # Iterate in batches over the training/test dataset.
            out = model(data.x, data.edge_index, data.batch)  
            pred = out.argmax(dim=1)  # Use the class with highest probability.
            correct += int((pred == data.y).sum())  # Check against ground-truth labels.
            true_labels += data.y.tolist()
            pred_labels += pred.tolist()
        return f1_score(true_labels, pred_labels, average='macro'), precision_score(true_labels, pred_labels, average='macro'), recall_score(true_labels, pred_labels, average='macro')


    for epoch in range(0, n_epoch):
        train()
        train_f1, train_pre, train_rec = test(train_loader)
        test_f1, test_pre, test_rec = test(test_loader)
        if epoch % 1 == 0:
            print(f'Epoch: {epoch:03d}, Train F1: {train_f1:.4f}, Test F1: {test_f1:.4f}')
    
    model.eval()
    dim_vec = torch.empty((0, n_hidden_channels), dtype=torch.float32)
    for data in all_data_loader:
        dim_x = model.dimReduce(data.x, data.edge_index, data.batch)
        dim_vec = torch.cat((dim_vec, dim_x), 0)
    return (dim_vec)

In [52]:
all_data_loader = DataLoader(dataset, batch_size=1, shuffle=False)
dim_vec = train_model(all_data_loader, train_loader, test_loader)

Epoch: 000, Train F1: 0.4013, Test F1: 0.4016


In [53]:
len(dataset)

2231

In [54]:
path_dir="results"

In [55]:
def runHUGPHISH(train_idx, test_idx):
    # stack_GNNs_graph = np.concatenate((np.array(phish_url_vectorizer), hyperlink_features),axis=1)
    stack_GNNs_graph = np.array(ngramUrl_manualHtml_features)
    for i in range(10):
        np.random.seed(i) 
        # train_idx_new = list(np.random.choice(list(range(n_samples)), int(0.8*n_samples), replace=False))
        # train_idx_new = list(set(train_idx_new).difference(test_idx))
        train_idx_new = list(np.random.choice(train_idx, int(0.8*len(train_idx)), replace=False))
        print(train_idx_new[:5])
        train_dataset_new = [dataset[idx] for idx in train_idx_new]
        train_loader_new = DataLoader(train_dataset_new, batch_size=8, shuffle=True)
        # test_dataset = [dataset[idx] for idx in test_idx]
        # test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
        n_hidden_channels = 6
        n_epoch = 1
        dim_vec_new = train_model(all_data_loader, train_loader_new, test_loader, n_hidden_channels, n_epoch)
        stack_GNNs_graph = np.concatenate((stack_GNNs_graph, dim_vec_new.detach().numpy()),axis=1)
    
    model_lgb2 = lgb.LGBMClassifier(verbose=-1)
    model_lgb2.fit(stack_GNNs_graph[train_idx], labels[train_idx])
    y_predict=model_lgb2.predict(stack_GNNs_graph[test_idx]) 
    f1 = f1_score(test_labels, y_predict, average='macro')
    pre = precision_score(test_labels, y_predict, average='macro')
    rec = recall_score(test_labels, y_predict, average='macro')
    print(f1, pre, rec)
    

In [56]:
runHUGPHISH(train_idx, test_idx)

[1010, 942, 1183, 963, 1422]
Epoch: 000, Train F1: 0.4012, Test F1: 0.4016
[1546, 851, 302, 108, 1719]
Epoch: 000, Train F1: 0.4017, Test F1: 0.4016
[612, 505, 502, 799, 1089]
Epoch: 000, Train F1: 0.3997, Test F1: 0.4016
[610, 889, 1767, 774, 361]
Epoch: 000, Train F1: 0.4007, Test F1: 0.4016
[674, 1753, 256, 956, 1061]
Epoch: 000, Train F1: 0.4044, Test F1: 0.4016
[120, 1467, 1266, 628, 1432]
Epoch: 000, Train F1: 0.4017, Test F1: 0.4016
[1435, 649, 88, 140, 702]
Epoch: 000, Train F1: 0.4064, Test F1: 0.4016
[58, 281, 1102, 452, 788]
Epoch: 000, Train F1: 0.4002, Test F1: 0.4016
[1566, 195, 1537, 1520, 1679]
Epoch: 000, Train F1: 0.4027, Test F1: 0.4016
[1525, 301, 412, 404, 607]
Epoch: 000, Train F1: 0.4039, Test F1: 0.4016
0.9117642082466202 0.9097306397306397 0.9139115646258503
